In [43]:
import pandas as pd

df = pd.read_json('Atlas_data_frame.json')


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   index                              112 non-null    object 
 1   id                                 112 non-null    int64  
 2   pageProps.projects.hits.hits.sort  112 non-null    int64  
 3   project_id                         112 non-null    int64  
 4   type                               112 non-null    object 
 5   regionShortCode                    111 non-null    object 
 6   developer                          112 non-null    object 
 7   name                               112 non-null    object 
 8   region                             112 non-null    object 
 9   polygonCoordinate                  112 non-null    object 
 10  square                             112 non-null    float64
 11  cidScreenShot                      112 non-null    object 

In [44]:
def remove_trailing_zeros(s):
  
    if s.startswith('list(c(') and s.endswith(')'):
       
        s = s[7:-1]
  
    list_of_strings = s.split(',')
   
    list_of_strings = [s.strip().lstrip('c(').rstrip(')') for s in list_of_strings]
    
    list_of_floats = list(map(float, list_of_strings))
   
    while list_of_floats and list_of_floats[-1] == 0.0:
        list_of_floats.pop()
   
    return ', '.join(map(str, list_of_floats))


df['polygonCoordinate'] = df['polygonCoordinate'].apply(remove_trailing_zeros)

In [47]:
import pandas as pd
import simplekml
import os

if not os.path.exists('kml'):
    os.makedirs('kml')

for index, row in df.iterrows():
    
    coords = list(map(float, row['polygonCoordinate'].split(',')))

   
    half = len(coords) // 2
    lats = coords[:half]
    longs = coords[half:]

   
    polygon_coords = list(zip(lats, longs))


    kml = simplekml.Kml()

    pol = kml.newpolygon(name="Polygon{}".format(index), outerboundaryis=polygon_coords)

    pol.style.polystyle.color = simplekml.Color.changealphaint(0, simplekml.Color.white)

    
    pol.style.linestyle.color = simplekml.Color.red

    kml.save(os.path.join("kml", "Polygon{}.kml".format(index)))

In [45]:
df.to_csv('OPF.csv', index=False)

In [29]:
df["polygonCoordinate"]

0      -97.9555704, -97.9555678, -97.9558387, -97.955...
1      79.4147657, 79.4147415, 79.4150232, 79.4151144...
2      34.5074755574286, 34.50678510184, 34.506173427...
3      -89.5362508149241, -89.5350081672492, -89.5349...
4      -89.528162, -89.5273918, -89.531951, -89.53207...
                             ...                        
107    -76.4360839189999, -76.435808104, -76.43591829...
108    -76.443557912, -76.4437644598984, -76.44401264...
109    -76.43563696, -76.4351364539999, -76.435444542...
110    -76.444705495, -76.445244465, -76.445595946999...
111    -76.439519735, -76.4398112629999, -76.4399827,...
Name: polygonCoordinate, Length: 112, dtype: object

In [18]:
import simplekml
import pandas as pd
import glob
import os


output_dir = "Kml_files"


if not os.path.exists(output_dir):
    os.makedirs(output_dir)


for index, row in df.iterrows():
    kml = simplekml.Kml()
  
    coordinates_list = row['polygonCoordinate'][7:-1].split(', ')
 
    coordinates_list = [coord.replace('c(', '').replace(')', '') for coord in coordinates_list]
  
    coordinates_list = [coord for coord in coordinates_list if coord]
  
    coord_length = len(coordinates_list) // 3
    
    longitudes = coordinates_list[:coord_length]
    latitudes = coordinates_list[coord_length:2*coord_length]
    altitudes = coordinates_list[2*coord_length:]

    coordinates = list(zip(map(float, longitudes), map(float, latitudes), map(float, altitudes)))
    if coordinates:  
        pol = kml.newpolygon(name="Polygon {}".format(index), outerboundaryis=coordinates)
        pol.style.polystyle.color = simplekml.Color.changealphaint(0, simplekml.Color.red)  
        pol.style.linestyle.color = simplekml.Color.red 
        kml.save(os.path.join(output_dir, "Polygon_{}.kml".format(index)))

kml_files = glob.glob(os.path.join(output_dir, '*.kml'))
print("Number of KML files: ", len(kml_files))

Number of KML files:  112


In [28]:



output_dir = "Kml_files"


if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for index, row in df.iterrows():
    kml = simplekml.Kml()
 
    coordinates_list = row['polygonCoordinate'][7:-1].split(', ')
 
    coordinates_list = [coord.replace('c(', '').replace(')', '') for coord in coordinates_list]
    mid_index = len(coordinates_list) // 2
    longitudes = coordinates_list[:mid_index]
    latitudes = coordinates_list[mid_index:-1]  
    coordinates = list(zip(map(float, latitudes), map(float, longitudes))) 
    if coordinates: 
        pol = kml.newpolygon(name="Polygon {}".format(index), outerboundaryis=coordinates)
        pol.style.polystyle.color = simplekml.Color.changealphaint(0, simplekml.Color.red)  
        pol.style.linestyle.color = simplekml.Color.red  
        kml.save(os.path.join(output_dir, "Polygon_{}.kml".format(index)))


kml_files = glob.glob(os.path.join(output_dir, '*.kml'))
print("Number of KML files: ", len(kml_files))

Number of KML files:  112


## VERRA DATA EXTRACTION

In [21]:
from pykml import parser
from os import path
import pandas as pd
import re 


with open('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST KE PD-VCS-011d App03a PA Plots 230701.kml', 'r') as kml_file:
    kml_content = parser.parse(kml_file).getroot().Document


ids = []
coordinates = []
extended_data_list = [] 


def parse_coordinates(coordinate_string):
    coordinate_list = re.split(r'\s+', coordinate_string.strip())
    return [(float(coordinate.split(',')[1]), float(coordinate.split(',')[0])) for coordinate in coordinate_list]


for folder in kml_content.Folder:
  
    for placemark in folder.Placemark:
        if hasattr(placemark, 'name'):
            id = placemark.name.text
        else:
            id = None
        if hasattr(placemark, 'ExtendedData'):
            extended_data = {}
            for data in placemark.ExtendedData.SchemaData.SimpleData:
                extended_data[data.attrib['name']] = data.text
            extended_data_list.append(extended_data)
        else:
            extended_data_list.append(None)
        if hasattr(placemark, 'Point'):
            coordinate = placemark.Point.coordinates.text.strip().split(',')
            longitude, latitude = list(map(float, coordinate))
            ids.append(id)
            coordinates.append([(latitude, longitude)])
        elif hasattr(placemark, 'LineString'):
            coordinate_string = placemark.LineString.coordinates.text
            ids.append(id)
            coordinates.append(parse_coordinates(coordinate_string))
        elif hasattr(placemark, 'MultiGeometry'):
            for polygon in placemark.MultiGeometry.Polygon:
                coordinate_string = polygon.outerBoundaryIs.LinearRing.coordinates.text
                ids.append(id)
                coordinates.append(parse_coordinates(coordinate_string))
        elif hasattr(placemark, 'Polygon'):
            coordinate_string = placemark.Polygon.outerBoundaryIs.LinearRing.coordinates.text
            ids.append(id)
            coordinates.append(parse_coordinates(coordinate_string))

df = pd.DataFrame({
    'ID': ids,
    'Coordinates': coordinates,
    'ExtendedData': extended_data_list 
})


df.to_csv('kenya_2.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST KE PD-VCS-011d App03a PA Plots 230701.kml'

In [53]:
from pykml import parser
from os import path
import pandas as pd

# Parse the KML file
with open('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/kml_files/03-Dingxi_Project boundary.kml', 'r') as kml_file:
    kml_content = parser.parse(kml_file).getroot().Document

# Initialize lists to hold the data
ids = []
geometries = []

# Iterate over the Placemarks in the KML file
for placemark in kml_content.Folder.Placemark:
    id = placemark.name.text
    geometry = []
    if hasattr(placemark, 'Point'):
        coordinates = placemark.Point.coordinates.text.strip().split(',')
        longitude, latitude = map(float, coordinates)
        geometry.append((latitude, longitude))
    elif hasattr(placemark, 'LineString'):
        coordinates = placemark.LineString.coordinates.text.strip().split(' ')
        for coordinate in coordinates:
            longitude, latitude = map(float, coordinate.split(','))
            geometry.append((latitude, longitude))
    ids.append(id)
    geometries.append(geometry)

# Save the data to a DataFrame
df = pd.DataFrame({
    'ID': ids,
    'Polygon_Geometry': geometries
})

# Write the DataFrame to a CSV file
df.to_csv('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/kml_files/03-Dingxi_Project boundary.csv', index=False)

AttributeError: no such child: {http://www.opengis.net/kml/2.2}name

In [ ]:
from pykml import parser
from os import path
import pandas as pd

# Parse the KML file
with open('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST KE PD-VCS-001d App03 PA Plots.kml', 'r') as kml_file:
    kml_content = parser.parse(kml_file).getroot().Document

# Initialize lists to hold the data
ids = []
geometries = []

# Iterate over the Placemarks in the KML file
for placemark in kml_content.Folder.Placemark:
    id = placemark.name.text
    coordinates = placemark.LineString.coordinates.text.strip().split(' ')
    geometry = []
    for coordinate in coordinates:
        longitude, latitude, altitude = map(float, coordinate.split(','))
        geometry.append((latitude, longitude))
    ids.append(id)
    geometries.append(geometry)

# Save the data to a DataFrame
df = pd.DataFrame({
    'ID': ids,
    'Polygon_Geometry': geometries
})

# Write the DataFrame to a CSV file
df.to_csv('Kenya_1.csv', index=False)

AttributeError: no such child: {http://www.opengis.net/kml/2.2}LineString

In [54]:
from pykml import parser
from os import path
import pandas as pd
from bs4 import BeautifulSoup


with open('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/kml_files/03-Dingxi_Project boundary.kml', 'r') as kml_file:
    kml_content = parser.parse(kml_file).getroot().Document


names = []
descriptions = []
coordinates = []


def parse_coordinates(coordinate_string):
    coordinate_list = coordinate_string.strip().split()
    return [(float(coordinate.split(',')[1]), float(coordinate.split(',')[0])) for coordinate in coordinate_list]

def parse_description(description):
    if description is None:
        return None
    soup = BeautifulSoup(description, 'html.parser')
    table = soup.find('table')
    data = {}
    for row in table.find_all('tr'):
        cells = row.find_all('td')
        if len(cells) > 1:
            key = cells[0].text.strip().replace(':', '')
            value = cells[1].text.strip()
            data[key] = value
    return data


for folder in kml_content.Folder:

    for placemark in folder.Placemark:
        name = placemark.name.text if hasattr(placemark, 'name') else None
        description = parse_description(placemark.description.text) if hasattr(placemark, 'description') and placemark.description.text is not None else None
        if hasattr(placemark, 'Point'):
            coordinate = parse_coordinates(placemark.Point.coordinates.text)
        elif hasattr(placemark, 'LineString'):
            coordinate = parse_coordinates(placemark.LineString.coordinates.text)
        else:
            coordinate = None
        names.append(name)
        descriptions.append(description)
        coordinates.append(coordinate)


df = pd.DataFrame({
    'Name': names,
    'Description': descriptions,
    'Coordinates': coordinates
})


df.to_csv('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/kml_files/03-Dingxi_Project boundary.csv', index=False)

In [56]:
from pykml import parser
from os import path
import pandas as pd
from bs4 import BeautifulSoup

# Parse the KML file
with open('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/kml_files/03-Dingxi_Project boundary.kml', 'r') as kml_file:
    kml_content = parser.parse(kml_file).getroot().Document


kcodes = []
plant_years = []
layers = []
paths = []
coordinates = []


def parse_coordinates(coordinate_string):
    coordinate_list = coordinate_string.strip().split()
    return [(float(coordinate.split(',')[0]), float(coordinate.split(',')[1])) for coordinate in coordinate_list]


for folder in kml_content.Folder:

    for placemark in folder.Placemark:
        kcode = placemark.ExtendedData.SchemaData.getchildren()[0].text
        plant_year = placemark.ExtendedData.SchemaData.getchildren()[1].text
        layer = placemark.ExtendedData.SchemaData.getchildren()[2].text
        path = placemark.ExtendedData.SchemaData.getchildren()[3].text
        coordinate = parse_coordinates(placemark.MultiGeometry.Polygon.outerBoundaryIs.LinearRing.coordinates.text)
        kcodes.append(kcode)
        plant_years.append(plant_year)
        layers.append(layer)
        paths.append(path)
        coordinates.append(coordinate)

df = pd.DataFrame({
    'kcode': kcodes,
    'PlantYear': plant_years,
    'layer': layers,
    'path': paths,
    'Coordinates': coordinates
})


df.to_csv('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/kml_files/03-Dingxi_Project boundary.csv', index=False)

In [64]:
from pykml import parser
import pandas as pd
from lxml import etree


kml_file_path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/kml_files/05-Zhanjiang AR_Project boundary.kml'


def parse_coordinates(coordinate_string):
    coordinate_list = coordinate_string.strip().split()
    return [(float(coordinate.split(',')[0]), float(coordinate.split(',')[1])) for coordinate in coordinate_list]


areas = []
coordinates = []


with open(kml_file_path, 'r') as kml_file:
    kml_content = parser.parse(kml_file).getroot()
    namespaces = {'kml': 'http://www.opengis.net/kml/2.2'}

    for placemark in kml_content.findall('.//kml:Placemark', namespaces):
       
        area_element = placemark.find('.//kml:SimpleData[@name="area"]', namespaces)
        if area_element is not None:
            area = area_element.text
            areas.append(float(area))
        else:
           
            areas.append(0)  
 
        coordinate_element = placemark.find('.//kml:coordinates', namespaces)
        if coordinate_element is not None and coordinate_element.text:
            coordinate_string = coordinate_element.text
            coordinate = parse_coordinates(coordinate_string)
            coordinates.append(coordinate)
        else:
           
            coordinates.append([])  


df = pd.DataFrame({
    'Area': areas,
    'Coordinates': coordinates
})


csv_file_path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/kml_files/05-Zhanjiang AR_Project boundary.csv'
df.to_csv(csv_file_path, index=False)

In [52]:
from lxml import etree as ET
from bs4 import BeautifulSoup
import pandas as pd

def parse_kml_to_csv(kml_file_path, csv_file_path):
    try:
        parser = ET.XMLParser(recover=True)  
        with open(kml_file_path, 'rb') as file:
            tree = ET.parse(file, parser=parser)  
            root = tree.getroot()
    except Exception as e:
        print(f"Error parsing the KML file: {e}")
        return

    namespaces = {'kml': 'http://www.opengis.net/kml/2.2'}  
    
    data = []
    
    for placemark in root.findall('.//kml:Placemark', namespaces):
        placemark_data = parse_placemark(placemark, namespaces)  
        data.append(placemark_data)
    
    try:
        df = pd.DataFrame(data)
        df.to_csv(csv_file_path, index=False)
    except Exception as e:
        print(f"Error saving data to CSV: {e}")

def parse_placemark(placemark, namespaces):  
    placemark_data = {}
    for field in ['name',"area_ha"]:
        element = placemark.find(f'.//kml:{field}', namespaces)  
        if element is not None:
            placemark_data[field] = element.text.strip()
  
    coordinates = placemark.find('.//kml:coordinates', namespaces)
    if coordinates is not None:
        placemark_data['coordinates'] = coordinates.text.strip()
    parse_description(placemark_data)
    return placemark_data

def parse_description(placemark_data):
    if 'description' in placemark_data:
        soup = BeautifulSoup(placemark_data['description'], 'html.parser')
        for tr in soup.find_all('tr')[1:]:
            td_tags = tr.find_all('td')
            if len(td_tags) == 2:
                key = td_tags[0].text.strip()
                value = td_tags[1].text.strip()
                placemark_data[key] = value

# Example usage
kml_file_path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Project areas ID738.kml'
csv_file_path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Project areas ID738.csv'
parse_kml_to_csv(kml_file_path, csv_file_path)